# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770919383066                   -0.53    9.0         
  2   -2.772145084478       -2.91       -1.30    1.0    189ms
  3   -2.772170148596       -4.60       -2.68    1.0    227ms
  4   -2.772170721691       -6.24       -4.09    2.0    220ms
  5   -2.772170723004       -8.88       -5.12    2.0    264ms
  6   -2.772170723015      -10.95       -5.88    2.0    227ms
  7   -2.772170723015      -12.63       -6.91    1.0    217ms
  8   -2.772170723015      -13.95       -7.57    2.0    259ms
  9   -2.772170723015   +  -14.27       -8.32    1.0    220ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770749689589                   -0.53    8.0         
  2   -2.772054641480       -2.88       -1.31    1.0    221ms
  3   -2.772082924247       -4.55       -2.55    1.0    190ms
  4   -2.7

1.7735577124311241

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770697901413                   -0.53    8.0         
  2   -2.772049665174       -2.87       -1.31    1.0    234ms
  3   -2.772082832794       -4.48       -2.59    1.0    189ms
  4   -2.772083384923       -6.26       -3.95    1.0    217ms
  5   -2.772083417716       -7.48       -4.63    2.0    270ms
  6   -2.772083417809      -10.03       -5.63    2.0    226ms
  7   -2.772083417811      -11.78       -6.43    2.0    271ms
  8   -2.772083417811      -13.68       -6.97    2.0    232ms
  9   -2.772083417811   +  -13.81       -8.52    1.0    246ms

Polarizability via ForwardDiff:       1.7725349699100676
Polarizability via finite difference: 1.7735577124311241
